In [ ]:
import pandas as pd
import os

def average_service_time(file_path):
    """
    Reads the 'Settings' and 'Games' sheets of the combined.xlsx at file_path,
    identifies which side (host/guest) the UCLA player is in each match,
    sums the durations of only those games where that side served,
    and returns the average serve-time per match as an "M:SS" string.
    """
    # Infer player name from the parent folder
    player = os.path.basename(os.path.dirname(file_path))
    
    # Load sheets
    xls      = pd.ExcelFile(file_path)
    settings = xls.parse('Settings')
    games    = xls.parse('Games')
    
    # Clean up Settings: get one Host/Guest pair per match file
    settings_clean = (
        settings
        .groupby('__source_file__', as_index=False)
        .agg({'Host Team':'first', 'Guest Team':'first'})
    )
    
    # Collect total serve-time (in seconds) per match
    serve_totals = []
    for _, row in settings_clean.iterrows():
        match_file = row['__source_file__']
        host       = row['Host Team']
        guest      = row['Guest Team']
        
        # Determine which side UCLA player is
        if host  == player:
            side = 'host'
        elif guest == player:
            side = 'guest'
        else:
            # Player not in this match
            continue
        
        # Filter games for this match & side
        dfm = games[
            (games['__source_file__'] == match_file) &
            (games['Server']           == side)
        ].copy()
        
        # Convert Duration to numeric, drop missing
        dfm['Duration'] = pd.to_numeric(dfm['Duration'], errors='coerce')
        dfm = dfm[dfm['Duration'].notna()]
        
        # Sum up total seconds for this match
        serve_totals.append(dfm['Duration'].sum())
    
    if not serve_totals:
        return None
    
    # Compute the average seconds, then format as M:SS
    avg_seconds = sum(serve_totals) / len(serve_totals)
    total = int(round(avg_seconds))
    mins, secs = divmod(total, 60)
    return f"{mins}:{secs:02d}"
    

# Example usage:
path = "/Users/eshaan/Desktop/Tennis Scouting/consulting-spring2025/data/mens/Rudy Quan/combined.xlsx"
avg = average_service_time(path)  
avg





'25:50'